## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
to_drop = ['EIN', 'NAME']
cleaned_df = application_df.drop(columns= to_drop)
cleaned_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_counts = cleaned_df.nunique()

# Display the results
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = cleaned_df["APPLICATION_TYPE"].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_type_counts[application_type_counts < 1000].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = cleaned_df["CLASSIFICATION"].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = classification_counts[classification_counts > 1]
print(classification_counts_gt1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < 900].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    cleaned_df['CLASSIFICATION'] = cleaned_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(cleaned_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'CLASSIFICATION', "ORGANIZATION", "AFFILIATION", "SPECIAL_CONSIDERATIONS", "USE_CASE", "INCOME_AMT"]
dummy_df = pd.get_dummies(cleaned_df, columns=categorical_columns)
dummy_cleaned =dummy_df.drop(columns = [ "INCOME_AMT_1M-5M", "INCOME_AMT_5M-10M", "INCOME_AMT_10M-50M", "INCOME_AMT_50M+"])

# Display the first few rows of the dummy DataFrame
dummy_cleaned.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,1,108590,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = dummy_cleaned.drop(columns=['IS_SUCCESSFUL'])
y = dummy_cleaned['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=52)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [45]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = 40, activation= "relu", input_dim= 47))

# Second hidden layer



nn.add(tf.keras.layers.Dense(units = 30, activation= "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation= "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 40)                1920      
                                                                 
 dense_30 (Dense)            (None, 30)                1230      


                                                                 
 dense_31 (Dense)            (None, 1)                 31        
                                                                 
Total params: 3181 (12.43 KB)
Trainable params: 3181 (12.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [49]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7383
Epoch 2/200
804/804 [==============================] - 1s 956us/step - loss: 0.5379 - accuracy: 0.7390
Epoch 3/200
804/804 [==============================] - 1s 946us/step - loss: 0.5383 - accuracy: 0.7389
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7376
Epoch 5/200
804/804 [==============================] - 1s 960us/step - loss: 0.5381 - accuracy: 0.7391
Epoch 6/200
804/804 [==============================] - 1s 961us/step - loss: 0.5381 - accuracy: 0.7384
Epoch 7/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5385 - accuracy: 0.7387
Epoch 8/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7394
Epoch 9/200
804/804 [==============================] - 1s 976us/step - loss: 0.5379 - accuracy: 0.7393
Epoch 10/200
804/804 [==============================] - 1s 1ms/step - loss: 0.538

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5774 - accuracy: 0.7257 - 173ms/epoch - 647us/step
Loss: 0.5774354338645935, Accuracy: 0.7257142663002014


In [38]:
# Export our model to HDF5 file
